In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Ellipse
import matplotlib.patches as patches
import librosa

%run manuallyEntered_graphing_functions.ipynb

In [2]:
#Use this for all adjustable values 
#input file paths here (include .csv):
video_path = r'\\10.25.16.5\yaochen\Active\DLC\Final_Models\Hanna\escape_behavior_HannaJachna_2024-07-8\memoryTest\coords_csv\Coordretest_M1_07-05-24_Hanna_videoDLC_resnet50_escape_behavior_2Jun13shuffle1_1030000_filtered_day.csv'
time_path = r'\\10.25.16.5\yaochen\Active\Hanna\Escape_behavior\Variation3_memoryTestAfter2Weeks\timestamp_07-05-24_Hanna\retest_M1_07-05-24_Hanna_timestamp0.csv'
audio_path = r'\\10.25.16.5\yaochen\Active\Hanna\Escape_behavior\Variation3_memoryTestAfter2Weeks\audio_07-05-24_Hanna\retest_M1_07-05-24_Hanna_audio.wav'

#input coordinates for the shelter and platform 
north_pixel = -13
south_pixel = 453
east_pixel = 577
west_pixel = 76
shelter_x_pixel = 324
shelter_y_pixel = 379

#input info from time_and_audio info notebook (either from audacity or bonsai)
stimulus_start = 32.149 #stimulus start time
escape_time = 36.0 #time mouse reaches shelter location

#set sigma for gaussian filter - look into how to decide 
sigma = 2 


In [3]:

#manually decide?
freezing_response = False

#does the mouse turn its head in the shortest angle to shelter
small_angle = True

In [4]:
#read in DLC data (video_path), convert all points to cm, and center platform at 0,0
#return converted coordinates for all in dataframe called coords, return new coordinates for the shelter and the x and y diam
#parameters: N, S, E, W, shelter_x, shelter_y, video path (pixel information and DLC data defined in cell 2)
shelter_x, shelter_y, x_diam, y_diam, coords = convert_data(north_pixel, south_pixel, east_pixel, west_pixel, shelter_x_pixel, shelter_y_pixel, video_path)

In [5]:
#replace any mouse skeleton points with likelihood < .9 with NA (stored in coords)
#coords dataframe is filled with NA when LT<.9, coords_scaled has interpolated data for NA values
#returns new df (coord_scaled) with low threshold values interpolated

#parameters: likelihood threshold and dataframe with DLC data
coord_scaled = coords
#filter_data(.9, coords)

In [6]:
#add column for time in seconds for both coords (na values) and coord_scaled (interpolated). stimulus at 0s.
#choose which stimulus to look at - 1 or 2 

#parameters: path to bonsai video timestamps, dataframe, audio path (stimulus timestamps from bonsai), which stimulus you want to look at
escape_estimate = audio_timing(stimulus_start, audio_path, time_path, escape_time, coord_scaled)
escape_estimate = audio_timing(stimulus_start, audio_path, time_path, escape_time,coords)
escape_estimate = escape_time - stimulus_start
print(escape_estimate)


3.850999999999999


In [7]:
#adds 3 new columns: 1 is an average of all points to use instead of the center of the mouse, another is averaged upper body (spine1, center) and the last is averaged head (nose, head, ears)
#more robust to use average of multiple points vs a single variable point (esp when looking at head angle bc of nose)

#parameters: dataframe 
average_points(coords)
average_points(coord_scaled)


In [8]:
#calculate displacement as 'displacement'

#parameters: converted shelter_x, converted shelter_y, dataframe
displacement(shelter_x, shelter_y, coord_scaled) 
displacement(shelter_x, shelter_y, coords) 

In [9]:
#calculate speed as 'speed' and change in displacement as 'delta_vector' (use delta vector for linearity calculations)

#parameters: dataframe 
total_distance = speed(coord_scaled) 
total_distance_coords = speed(coords) 


In [10]:
#calculate head angle as 'head_angle'

#parameters: converted shelter_x, converted shelter_y, dataframe
angle(shelter_x, shelter_y, coord_scaled) 
angle(shelter_x, shelter_y, coords) 

In [11]:
#calculate head angle speed as 'angle_speed'
#haven't assessed
#parameters: dataframe
angle_speed(coord_scaled) 
angle_speed(coords) 


In [12]:
#determines more precise time for escape - either when mouse reaches shelter border OR gets closest

#parameters: converted shelter x, converted shelter y, estimate for shelter reaching time, dataframe
#escape_done = escape_timeframe(shelter_x, shelter_y, escape_estimate, coord_scaled)
#print(escape_done)

print(coord_scaled)

print(coord_scaled.head())

      Unnamed: 0  frames     nose_x     nose_y  nose_likelihood     head_x  \
0              0       0  10.503180 -29.410929         0.000289  19.340573   
1              1       1  10.503180 -29.410929         0.000291  19.342301   
2              2       2  10.503180 -29.410929         0.000308  19.351928   
3              3       3  10.488726 -29.410929         0.000391  19.364684   
4              4       4  10.467279 -29.409728         0.000368  19.391040   
...          ...     ...        ...        ...              ...        ...   
1146        1146    1146  16.159343  18.834569         0.834054  18.193722   
1147        1147    1147  16.156690  18.815434         0.809423  18.193722   
1148        1148    1148  16.133663  18.811052         0.802627  18.188059   
1149        1149    1149  16.126243  18.729898         0.911245  17.346379   
1150        1150    1150  16.089465  18.692606         0.891997  17.338703   

         head_y  head_likelihood  earRight_x  earRight_y  ...  

In [13]:
#finds linearity ratio 

#parameters: shelter reaching time found in cell 12, dataframe
ratio, initial_displacement, total_distance = linearity_ratio(0, escape_estimate, coord_scaled)

IndexError: single positional indexer is out-of-bounds

In [ ]:
#determine whether response is classified as an escape - look at time, linearity, and shelter reach

#parameters: shelter reaching time, converted shelter x, converted shelter y, dataframe, desired percent shelter expansion, time, linearity cutoffs
#escape = escape_response(escape_done, shelter_x, shelter_y, coord_scaled, 30, 12, 1.3)
#print(escape)

In [ ]:
#can edit code to use this dataframe in above graphs, have this defined to pass into compiled mice. Apply filter to displacement, speed, angle, angle speed (angle speed not accurate)

#parameter: sigma, dataframe
coord_scaled_smooth = smooth_curve(sigma, coord_scaled)

In [ ]:
#parameters: converted shelter x, converted shelter y, converted x diameter, converted y diameter, start time, stop time, dataframe
#if moves away from shelter or freezes use escape estimate or you get an empty df 

trajectories(shelter_x, shelter_y, x_diam, y_diam, 0, escape_estimate, coord_scaled)
print("The displacement of the mouse from the stimulus onset to when it reaches the shelter is " + str(initial_displacement) + " cm.") 
print("The total distance that the mouse travels during the stimulus time is  " + str(total_distance) + " cm.") 
print("Mouse reaches point closest to prior shelter location with a linearity ratio of " + str(ratio) + ".") 

In [ ]:
#trajectory for 2 minutes of start of P2 video, trajectory 1 minute before and through escape
#parameters: converted shelter x, converted shelter y, converted x diam, converted y diam, shelter reaching time, dataframe
#if moves away from shelter or freezes use escape estimate or you get an empty df 

before_shelter_trajectory(shelter_x, shelter_y, x_diam, y_diam, escape_estimate, coord_scaled)

In [ ]:
#plot intermediate steps - raw and with gaussian filter

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

#parameters: ax, start time, stop time, dataframe, apply filter?, overlay filter and raw?, for vertical alignment? (includ labels or not), sigma value
speed_figure(ax1, -4, 8, coord_scaled, False, True, False, sigma)
speed_figure(ax2, -4, 8, coord_scaled, True, False, False,sigma)
speed_figure(ax3, -4, 8, coord_scaled, False, False, False,sigma)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

displacement_figure(ax1, -4, 8, coord_scaled, False, True, False,sigma)
displacement_figure(ax2, -4, 8, coord_scaled, True, False, False,sigma)
displacement_figure(ax3, -4, 8, coord_scaled, False, False, False,sigma)

#based on the timeframe, it appears that the mouse enters shelter zone within 3 seconds of stimulus start time but this graph is not showing this... go back and look into this.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(25, 6))

head_angle_figure(ax1, -4, 8, coord_scaled, False, True, False,sigma)
head_angle_figure(ax2, -4, 8, coord_scaled, True, False, False,sigma)
head_angle_figure(ax3, -4, 8, coord_scaled, False, False,False, sigma)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize = (4,10))

displacement_figure(ax1, -3, 6, coord_scaled, True, False, True, sigma)
speed_figure(ax2, -3, 6, coord_scaled, True, False, True, sigma)
head_angle_figure(ax3, -3, 6, coord_scaled, True, False, True, sigma)

plt.tight_layout()

In [ ]:
#normalized overlay of displacement, speed, head angle
#parameters: start time, stop time, dataframe
combined_graphs(-3, 8, coord_scaled_smooth)

In [ ]:
#likelihood_data, likelihood_minus_nose, likelihood_center = likelihood_check(0, 8, .9, coords)

In [ ]:
#new dataframe (rotate) with rotated coordinates and shelter at 0, 25

#parameters: converted shelter x, converted shelter y, start time, stop time, dataframe
#if moves away from shelter or freezes use escape estimate or you get an empty df 

rotate, shelter_rotate_x, shelter_rotate_y = align_shelter(shelter_x, shelter_y, 0, escape_estimate, coord_scaled)

In [ ]:
#visualize rotation above 
fig = plt.figure()  #create figure to fill in
ax = plt.axes()
    
    #make x and y scales equal - present as a circle


    #plot the center x and y coordinates
x = rotate['rotated_x']
y = rotate['rotated_y']

ax.text(shelter_rotate_x,shelter_rotate_y, 'S', color = 'black', size = 10)
shelter = patches.Rectangle(((shelter_rotate_x - 5.25), (shelter_rotate_y-6.25)), 10.5,12.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1)
ax.add_patch(shelter)

ax.set_aspect('equal')
ax.plot(x,y, color = 'blue', linewidth = 1)


In [ ]:
#try plotting all of P1s trajectory 